In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import sys
import os
sys.path.append(os.path.abspath(".."))  
sys.path.append(os.path.abspath("./src")) 



data_path = "../data/"
output_path = '../output/'
audio_path = "../audios/"

dataset ='Automatic_Labeling_Kyoogu/'
#dataset ='Dataset_1/'
#dataset ='Dataset_2/'

Full_DB_file = 'Labeled_Data_With_FCCs.xlsx'
Reduced_DB_file = 'Reduced_dataframe_FA2.xlsx'
isolated_observations_file = 'isolated_observations.xlsx'


df_full = pd.read_excel(data_path+dataset+Full_DB_file)
df_reduced = pd.read_excel(data_path+dataset+Reduced_DB_file,index_col=0)
df_isolated_observations = pd.read_excel(output_path+dataset+isolated_observations_file,index_col=0)

columns_to_keep =['File','Specie ID','Start','End','FminVoc','FmaxVoc']
df_filtered = df_full.loc[df_reduced.index, columns_to_keep]
df_filtered['Original Index'] = df_filtered.index
df_filtered = df_filtered.sort_values(by='File').reset_index(drop=True)
df_filtered


,File,Specie ID,Start,End,FminVoc,FmaxVoc,Original Index
0,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,55.2,57.199,55.2,3000,1093
1,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,8.8,10.799,8.8,3000,1089
2,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,0.0,2.399,0.0,3000,1088
3,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,50.4,52.399,50.4,3000,1092
4,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,36.8,38.799,36.8,3000,1091
...,...,...,...,...,...,...,...
1377,SMA03411_20210607_200000.wav,Dendropsophus_microcephalus,24.8,27.199,24.8,7200,524
1378,SMA03411_20210608_200000.wav,Dendropsophus_microcephalus,58.0,59.999,58.0,7200,536
1379,SMA03411_20210608_200000.wav,Dendropsophus_microcephalus,54.0,55.999,54.0,7200,535
1380,SMA03411_20210608_200000.wav,Dendropsophus_microcephalus,4.8,6.799,4.8,7200,534


In [35]:
# Rename the columns
columns_to_rename = {
    'Start': 'Begin Time (s)',
    'End': 'End Time (s)',
    'FminVoc': 'Low Freq (Hz)',
    'FmaxVoc': 'High Freq (Hz)'
}
df_filtered = df_filtered.rename(columns=columns_to_rename)

# Initialize the 'review' column in df_filtered with default values of 0
df_filtered['review'] = 0


# Set the 'review' column to 1 for rows where indices match those in df_isolated_observations
df_filtered.loc[df_filtered.index.isin(df_isolated_observations.index), 'review'] = 1


df_filtered.to_excel(output_path + dataset + "Labeled_Data_to_Review.xlsx", index=False)
df_filtered


,File,Specie ID,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Original Index,review
0,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,55.2,57.199,55.2,3000,1093,0
1,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,8.8,10.799,8.8,3000,1089,0
2,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,0.0,2.399,0.0,3000,1088,0
3,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,50.4,52.399,50.4,3000,1092,0
4,SMA03126_20210323_050000.wav,Leptodactylus_fuscus,36.8,38.799,36.8,3000,1091,0
...,...,...,...,...,...,...,...,...
1377,SMA03411_20210607_200000.wav,Dendropsophus_microcephalus,24.8,27.199,24.8,7200,524,1
1378,SMA03411_20210608_200000.wav,Dendropsophus_microcephalus,58.0,59.999,58.0,7200,536,1
1379,SMA03411_20210608_200000.wav,Dendropsophus_microcephalus,54.0,55.999,54.0,7200,535,0
1380,SMA03411_20210608_200000.wav,Dendropsophus_microcephalus,4.8,6.799,4.8,7200,534,1


In [36]:
# Create a dictionary to store the DataFrames
dataframes_dict = {}


# Iterate over the unique values in the 'File' column
for file_value in df_filtered['File'].unique():
    # Filter the original DataFrame for the current unique value
    df_filtered_copy = df_filtered[df_filtered['File'] == file_value].copy()
    
    # Add new columns
    df_filtered_copy['Selection'] = range(1, len(df_filtered_copy) + 1)
    df_filtered_copy['View'] = 'Spectrogram 1'
    df_filtered_copy['Channel'] = '1'

    # Reorder the columns
    columns_order = ['File', 'Selection', 'View', 'Channel'] + [col for col in df_filtered_copy.columns if col not in ['File', 'Selection', 'View', 'Channel']]
    df_filtered_copy = df_filtered_copy[columns_order]
    
    # Drop the 'File' column
    df_filtered_copy = df_filtered_copy.drop(columns=['File'])
    
    # Store the filtered DataFrame in the dictionary
    dataframes_dict[file_value] = df_filtered_copy
    
    # Save the DataFrame to a .txt file
    df_filtered_copy.to_csv(output_path + dataset + f"/SelectionTables/{file_value}.txt", sep='\t', index=False)

# Display the number of DataFrames created and the names of the keys in the dictionary
print(f"{len(dataframes_dict)} DataFrames have been created.")
print("The names of the keys in the dictionary are:", list(dataframes_dict.keys()))

# Display an example of the modified DataFrames
for key in list(dataframes_dict.keys())[:1]:  # Display only the first DataFrame as an example
    print(f"DataFrame for the file '{key}':")
    print(dataframes_dict[key].head())


123 DataFrames have been created.
The names of the keys in the dictionary are: ['SMA03126_20210323_050000.wav', 'SMA03126_20210323_051000.wav', 'SMA03126_20210323_195000.wav', 'SMA03126_20210323_200000.wav', 'SMA03126_20210327_072000.wav', 'SMA03126_20210609_182000.wav', 'SMA03126_20210611_175000.wav', 'SMA03126_20210611_200000.wav', 'SMA03175ICP_20210324_170000.wav', 'SMA03175ICP_20210326_052000.wav', 'SMA03175ICP_20210326_181000.wav', 'SMA03175ICP_20210607_073000.wav', 'SMA03175ICP_20210608_182000.wav', 'SMA03210_20210320_054000.wav', 'SMA03210_20210320_055000.wav', 'SMA03210_20210320_075000.wav', 'SMA03210_20210320_080000.wav', 'SMA03210_20210321_171000.wav', 'SMA03210_20210326_175000.wav', 'SMA03210_20210326_180000.wav', 'SMA03247_20210322_180000.wav', 'SMA03247_20210322_181000.wav', 'SMA03247_20210323_050000.wav', 'SMA03247_20210324_172000.wav', 'SMA03247_20210325_054000.wav', 'SMA03247_20210325_055000.wav', 'SMA03247_20210325_060000.wav', 'SMA03247_20210603_180000.wav', 'SMA03247

In [37]:
df_filtered['type'] = ''
df_filtered['Reason'] = ''

# Filter the original DataFrame for the current unique value
df_filtered_copy = df_filtered.copy()
df_filtered_copy['offset'] = 0

# Initialize offset multiplier and flag for the first file
current_offset = 0
first_file = True

# Iterate through the DataFrame and adjust times and offsets based on changes in 'File'
previous_file = None
for index, row in df_filtered_copy.iterrows():
    if row['File'] != previous_file:
        if not first_file:  # Only increase offset after the first file
            current_offset += 60
        first_file = False
    df_filtered_copy.at[index, 'offset'] = current_offset
    df_filtered_copy.at[index, 'Begin Time (s)'] += current_offset
    df_filtered_copy.at[index, 'End Time (s)'] += current_offset
    previous_file = row['File']


# Add new columns
df_filtered_copy['Selection'] = range(1, len(df_filtered_copy) + 1)
df_filtered_copy['View'] = 'Spectrogram 1'
df_filtered_copy['Channel'] = '1'

# Reorder the columns
columns_order = ['File', 'Selection', 'View', 'Channel'] + [col for col in df_filtered_copy.columns if col not in ['File', 'Selection', 'View', 'Channel']]
df_filtered_copy = df_filtered_copy[columns_order]

# Specify the output file path
if dataset == 'Dataset_1/':
    selection_table_file = 'All_sounds_subsetA.txt'
elif dataset == 'Dataset_2/':
    selection_table_file = 'All_sounds_subsetB.txt'
elif dataset == 'Automatic_Labeling_Kyoogu/':
    selection_table_file = 'All_sounds_subsetC.txt'

# Save the DataFrame to a .txt file
df_filtered_copy.to_csv(output_path + dataset + "/SelectionTables/"+ selection_table_file, sep='\t', index=False)



In [38]:

# Extract unique values from the 'File' column
unique_files = df_filtered['File'].unique()

# Specify the output file path
if dataset == 'Dataset_1/':
    list_file = 'subsetA.txt'
elif dataset == 'Dataset_2/':
    list_file = 'subsetB.txt'
elif dataset == 'Automatic_Labeling_Kyoogu/':
    list_file = 'subsetC.txt'
    dataset = 'Dataset_1/'

list_path = audio_path+dataset+'list_fie_'+list_file

# Write each value in unique_files to a new line in the output file
with open(list_path, 'w') as file:
    for file_name in unique_files:
        file.write(f"{file_name}\n")

print(f"File saved to {list_path}")

File saved to ../audios/Dataset_1/list_fie_subsetC.txt
